![](https://www.cranfield.ac.uk/~/media/Images/mastheads/masthead-bix-mh344x810.ashx?h=344&la=en&mw=810&w=810&hash=4AD63583A5EA5BF7E5E561A1C62A8F7879F36571)

# Exploratory Data Analysis

> Doing statistics is like doing crosswords except that one cannot know for sure whether one has found the solution.

<footer>~  John W. Tukey</footer>

![break](assets/agenda.png)

### Agenda

* Deal with Missing Data
* Explore Data by Visualisations

![break](assets/theory.png)

### Motivating Example... Catching Cheaters

![](assets/cplot.Rout-0.png)

[Source](http://jd-mathbio.blogspot.ca/2015/02/finding-cheaters-using-multiple-choice.html?utm_source=marketo&utm_medium=email&utm_campaign=DA-NL-202&mkt_tok=3RkMMJWWfF9wsRokvq3MZKXonjHpfsX%2B7%2BooW6Gg38431UFwdcjKPmjr1YEETcB0aPyQAgobGp5I5FEOS7PYS6V6t6EOUg%3D%3D)

## The Goals of Exploratory Data Analysis

1. Summarize the main characteristics of datasets with Summary Statistics (Initial Data Analysis)
1. Visually inspect the structure and nature of the data
1. Find what the data can tell us beyond the formal modeling or hypothesis testing task.

#### Example : Measurement Precision

![](assets/rounding1.png)

![](assets/rounding2.png)

![](assets/rounding3.png)

### Let the visuals speak to you

Always explore your data visually. Whatever specific hypothesis you have when you go out to collect data is likely to be worse than any of the hypotheses you’ll form after looking at just a few simple visualizations of that data.

#### Load `ggplot` and plot graphs inline

In [ ]:
%%sh
pip install ggplot

In [ ]:
from ggplot import *
import pandas as pd
import warnings

warnings.filterwarnings("ignore",category=DeprecationWarning)

%matplotlib inline

### How are zipcodes distributed in the US?

In [ ]:
zipcodes = pd.read_csv("../data/zipcodes.csv")
zipcodes

In [ ]:
ggplot(zipcodes, aes(x = 'zip', y = 'latitude')) + \
    geom_point() + \
    ggtitle("(Latitude, Zipcode) Scatterplot")

In [ ]:
ggplot(zipcodes, aes(x = 'zip', y = 'longitude')) + \
    geom_point() + \
    ggtitle("(Longitude, Zipcode) Scatterplot")

In [ ]:
ggplot(zipcodes, aes(x = 'longitude', y = 'latitude', color = 'zip')) + \
    geom_point() + \
    ggtitle("(Longitude, Latitude) Heatmap")

In [ ]:
ggplot(zipcodes[zipcodes['longitude'] < 0], aes(x = 'longitude', y = 'latitude', color = 'zip')) + \
    geom_point() + \
    ggtitle("(Longitude, Latitude) Heatmap")

### Objectives

* **Quality of measurements**
* **Quality of data**
* **Initial transformations**
* **Univariate statistics** (single variable)
* **Bivariate associations** (correlations)
* **Multivarite patterns** (analysis)

### Missing Data

In this section, we will discuss missing (also referred to as **NA**) values in pandas.

#### When / why does data become missing?

Some might quibble over our usage of missing. By “missing” we simply mean null or “not present for whatever reason”. Many data sets simply arrive with missing data, either because it exists and was not collected or it never existed. For example, in a collection of financial time series, some of the time series might start on different dates. Thus, values prior to the start date would generally be marked as missing.

As data comes in many shapes and forms, pandas aims to be flexible with regard to handling missing data. While `NaN` is the default missing value marker for reasons of computational speed and convenience, we need to be able to easily detect this value with data of different types: floating point, integer, boolean, and general object. In many cases, however, the Python `None` will arise and we wish to also consider that “missing” or “null”.

* NaN and None (in object arrays) are considered missing by the isnull and notnull functions. inf and -inf are no longer considered missing by default.
* To make detecting missing values easier (and across different array dtypes), pandas provides the `isnull()` and `notnull()` functions, which are also methods on `Series` objects:

In [ ]:
import pandas as pd
import numpy as np
from random import sample

data = range(10)

rows = sample(data, 6)
rows = [2, 3, 6, 8, 5, 7]

df = pd.DataFrame(data, columns=['data'])

df['fulldata'] = data

df.data.ix[rows[:3]] = None
df.data.ix[rows[3:]] = np.nan

In [ ]:
df

In [ ]:
pd.isnull(df.data)
# same as
df.data.isnull()

In [ ]:
pd.notnull(df.data)
# same as
df.data.notnull()

#### Calculations with missing data

The descriptive statistics and computational methods in pandas are all written to account for missing data

* When summing data, `NA` (missing) values will be treated as zero
* If the data are all `NA`, the result will be `NA`
* Methods like `cumsum` and `cumprod` ignore `NA` values, but preserve them in the resulting arrays

In [ ]:
df

In [ ]:
df['data'].sum()

In [ ]:
df.mean(0)

In [ ]:
df.cumsum()

### Cleaning / filling missing data

pandas objects are equipped with various data manipulation methods for dealing with missing data

#### Filling missing values: `fillna()`

The `fillna` function can “fill in” NA values with non-null data in a couple of ways, which we illustrate:

##### Replace NA with a scalar value

In [ ]:
df

In [ ]:
df.fillna(0)

In [ ]:
df['data'].fillna('missing')

#### Fill gaps forward or backward

We can propagate non-null values forward or backward:

In [ ]:
df.fillna(method='pad')

#### Limit the amount of filling

If we only want consecutive gaps filled up to a certain number of data points, we can use the limit keyword:

In [ ]:
df.fillna(method='pad', limit=1)

These are the available filling methods:

| Method | Action |
|--------|--------|
| pad / ffill	Fill | values forward |
| bfill / backfill | Fill values backward |

With time series data, using pad/ffill is extremely common so that the “last known value” is available at every time point.

The ffill() function is equivalent to fillna(method='ffill') and bfill() is equivalent to fillna(method='bfill')

You can also fillna using a `dict` or `Series` that is alignable. The labels of the `dict` or `index` of the `Series` must match the columns of the frame you wish to fill. The use case of this is to fill a DataFrame with the mean of that column.

In [ ]:
df.fillna(df.mean())
df.data

### Dropping axis labels with missing data: dropna

You may wish to simply exclude labels from a data set which refer to missing data. To do this, use the dropna method:

In [ ]:
df

In [ ]:
df.dropna()

In [ ]:
df.dropna(axis=1)

For a full overview of the API, see the [documentation](http://pandas.pydata.org/pandas-docs/dev/api.html#api-dataframe-missing)

### Interpolation

Both Series and Dataframe objects have an interpolate method that, by default, performs linear interpolation at missing datapoints.

This is where datatypes become important again. Notice the `dtype` of the 'data' column.

In [ ]:
df.data

First we need to get rid of the 'None' values, as this makes it into the loosest 'object' datatype. Using NaN we can use the float dataetime..

In [ ]:
df.isnull()

In [ ]:
df[df.isnull()] = np.nan
df

use `astype` to coerce a column into a specific datatype.

In [ ]:
df.data.astype('float')

In [ ]:
df.data = df.data.astype(np.float)

In [ ]:
df.data

In [ ]:
df.interpolate()

For a floating-point index, use method='values'

In [ ]:
dfxx = df.ix[[0,2,5,9]]

In [ ]:
dfxx

In [ ]:
dfxx.interpolate()

In [ ]:
dfxx.interpolate(method='values')

As a practical review of what's discussed here see this [summary](http://manishamde.github.io/blog/2013/03/07/pandas-and-python-top-10/#missing)

### Tips

Food servers’ tips in restaurants may be influenced by many factors, including the nature of the restaurant, size of the party, and table locations in the restaurant. Restaurant managers need to know which factors matter when they assign tables to food servers. For the sake of staff morale, they usually want to avoid either the substance or the appearance of unfair treatment of the servers, for whom tips (at least in restaurants in the United States) are a major component of pay.

In one restaurant, a food server recorded the following data on all customers they served during an interval of two and a half months in early 1990. The restaurant, located in a suburban shopping mall, was part of a national chain and served a varied menu. In observance of local law the restaurant offered seating in a non-smoking section to patrons who requested it. Each record includes a day and time, and taken together, they show the server’s work schedule.

    Source: Bryant, P. G. and Smith, M. A. (1995), Practical Data Analysis:
    Case Studies in Business Statistics, Richard D. Irwin Publishing, Homewood, IL.

**What are the factors that affect tipping behavior?**

#### Load the Dataset

In [ ]:
from ggplot import *

tips = pd.read_csv('../data/tips.csv')

#### Inspect the variables

In [ ]:
tips.describe()

In [ ]:
# info() is more informative than 'describe()' as
# it gives you the datatypes length of dataset

tips.info()

In [ ]:
# tips['smoker'].value_counts()
# tips[['smoker']].value_counts()

tips.index
# tips.ix[2:5, :'day']
tips.iloc[:,:]

In [ ]:
# Look at a random sample of the rows
tips.ix[sample(tips.index,10)]

In [ ]:
# Inspect head and tail of the dataset

In [ ]:
tips.head()

In [ ]:
tips.tail()

We look first at the variable of greatest interest to the waiter: tip. A common graph for looking at a single variable is the histogram, where data values are binned and the count is represented by a rectangular bar.

### Tip

Start your exploration of a single parameter - usually you'd pick the more important one for your research question.

In [ ]:
ggplot(tips, aes(x='tip')) + geom_histogram(binwidth=1)

In [ ]:
# Draw with black outline, yellow fill
ggplot(tips, aes(x='tip')) + geom_histogram(binwidth=1, colour="black", fill="yellow")

In [ ]:
# Density curve
ggplot(tips, aes(x='tip')) + geom_density()

In [ ]:
# Histogram overlaid with kernel density curve
ggplot(tips, aes(x='tip')) + \
    geom_histogram(aes(y='..density..'),      # Histogram with density instead of count on y-axis
                   binwidth=0.1,
                   colour="black", fill="yellow")

In [ ]:
# Add a line for the mean:
avg_tips = np.mean(tips.tip)

ggplot(tips, aes(x='tip')) + \
    geom_histogram(aes(y='..density..'),      # Histogram with density instead of count on y-axis
                   binwidth=1,
                   colour="black",
                   fill="yellow") + \
    geom_vline(aes(xintercept=avg_tips),
               color="red",
               linetype="dashed",
               size=2)

The distribution appears to be unimodal; that is, it has one peak, the bar representing the tips greater than \$1.50 and less than or equal \$2.50. There are
very few tips of \$1.50 or less. The number of larger tips trails off rapidly, which suggests that this is not a very expensive restaurant.

The conclusions drawn from a histogram are often influenced by the choice of bin width, which is a parameter of the _graph_ and not of the _data_.


#### Tip 2
Experiment with different bin-widths for you histograms - at low granulairty, you'll get a better idea of the overall distribution, at higher levels of granulairty you can inspect whether any 'clipping' is occuring. In other words, **a large bin width smooths out the graph and shows rough or global trends, whereas a smaller bin width highlights more local features**.

In [ ]:
# Draw with black outline, yellow fill, binwidth at $0.10
p01 = ggplot(tips, aes(x='tip')) + geom_histogram(binwidth=0.1, colour="black", fill="yellow")

# Draw with black outline, yellow fill, binwidth at $0.50
p05 = ggplot(tips, aes(x='tip')) + geom_histogram(binwidth=0.5, colour="black", fill="yellow")

# Draw with black outline, yellow fill, binwidth at $1
p1 = ggplot(tips, aes(x='tip')) + geom_histogram(binwidth=1, colour="black", fill="yellow")

# Draw with black outline, yellow fill, binwidth at $1
p4 = ggplot(tips, aes(x='tip')) + geom_histogram(binwidth=4, colour="black", fill="yellow")

In [ ]:
p05

In [ ]:
p1

In [ ]:
p4

This shows a histogram with a smaller bin width, 10c. At the smaller bin width the shape is multimodal, and it is clear that there are large peaks at the full
dollars and smaller peaks at the half dollar. This shows that the customers tended to round the tip to the nearest fifty cents or dollar.

So far we have not addressed the primary question: What relationships exist between tip and the other variables? Since the tip is usually calculated
based on the bill, it is natural to look first at a graph of tip and bill. A common graph for looking at a pair of continuous variables is the scatterplot.


### Scatterplot

In [ ]:
# GET EQUATION AND R-SQUARED AS STRING
# SOURCE: http://goo.gl/K4yh
tb = tips['totbill']
tr = tips['totbill'] * 0.18

ggplot(aes(x = 'totbill', y = 'tip'), data = tips) + \
    geom_line(aes(tb, tr), colour = "red", size = 1,) + \
    geom_point(colour = "black", size = 5)

We see that the variables are highly correlated ($r$ = 0.68), which confirms that tip is calculated from the bill. We have added a line representing a tip rate of 18\%. Disappointingly for the waiter, there are many more points below the line than above it: There are many more “cheap tippers” than generous tippers. There are a couple of notable exceptions, especially one party who gave a \$5.15 tip for a \$7.25 bill, which works out to a tip rate of about 70%.

**Histogram and density plots with multiple groups**

In [ ]:
# Overlaid histograms
ggplot(tips, aes(x='tip', fill='sex')) + \
    geom_histogram(binwidth=1, alpha=.5, position="identity")

In [ ]:
# Density plots
ggplot(tips, aes(x='tip', colour='sex')) + geom_density()

In [ ]:
# Density plots with semi-transparent fill
ggplot(tips, aes(x='tip', fill='sex')) + geom_density(alpha=.3)

We said earlier that an essential aspect of data visualization is capturing relationships among many variables: three, four, or even more. This dataset,
simple as it is, illustrates the point. Let us ask, for example, how a third variable such as sex affects the relationship between tip and bill. As sex is
categorical with two levels (i.e., binary), it is natural to divide the data into female and male payers and to generate two scatterplots of tip vs. bill. Let us go even further by including a fourth variable, smoking, which is also binary. We now divide the data into four parts and generate the four scatterplots.

In [ ]:
!sudo /home/io/.tools/anaconda/bin/pip install rpy2

In [ ]:
import rpy2
import pandas as pd
%load_ext rpy2.ipython

In [ ]:
%%R
# The R version of this code

# install.packages('ggplot2')
# install.packages('gridExtra')
library('ggplot2')
library('gridExtra')
tips <- read.csv('../data//tips.csv')

# GET EQUATION AND R-SQUARED AS STRING
# SOURCE: http://goo.gl/K4yhz

plt_cor <- function(subs,x,y) {
  plot_cor <- cor(subs[[x]],subs[[y]])
  return(round(plot_cor,2))
}

p1 <- ggplot(data = subset(tips, (sex == 'M' & smoker == 'No')), aes(x = totbill, y = tip)) +
        geom_point(colour = "black", size = 5) +
        geom_line(colour = "red", size = 1, aes(totbill, totbill*0.18)) +
        geom_text(aes(45, 7 ,label = '18%', vjust = -1)) +
        geom_text(aes(10, 9 ,label = paste('r', plt_cor(subset(tips, (sex == 'M' & smoker == 'No')),'totbill','tip')), vjust = -1)) +
        ggtitle("Male Non-Smokers") +
        xlim(1,50) + ylim(1,10)

p2 <- ggplot(data = subset(tips, (sex == 'M' & smoker == 'Yes')), aes(x = totbill, y = tip)) +
        geom_point(colour = "black", size = 5) +
        geom_line(colour = "red", size = 1, aes(totbill, totbill*0.18)) +
        geom_text(aes(45, 7 ,label = '18%', vjust = -1)) +
        geom_text(aes(10, 9 ,label = paste('r', plt_cor(subset(tips, (sex == 'M' & smoker == 'Yes')),'totbill','tip')), vjust = -1)) +
        ggtitle("Male Smokers") +
        xlim(1,50) + ylim(1,10)


p3 <- ggplot(data = subset(tips, (sex == 'F' & smoker == 'No')), aes(x = totbill, y = tip)) +
        geom_point(colour = "black", size = 5) +
        geom_line(colour = "red", size = 1, aes(totbill, totbill*0.18)) +
        geom_text(aes(45, 7 ,label = '18%', vjust = -1)) +
        geom_text(aes(10, 9 ,label = paste('r', plt_cor(subset(tips, (sex == 'F' & smoker == 'No')),'totbill','tip')), vjust = -1)) +
        ggtitle("Female Non-Smokers") +
        xlim(1,50) + ylim(1,10)


p4 <- ggplot(data = subset(tips, (sex == 'F' & smoker == 'Yes')), aes(x = totbill, y = tip)) +
        geom_point(colour = "black", size = 5) +
        geom_line(colour = "red", size = 1, aes(totbill, totbill*0.18)) +
        geom_text(aes(45, 7 ,label = '18%', vjust = -1)) +
        geom_text(aes(10, 9 ,label = paste('r', plt_cor(subset(tips, (sex == 'F' & smoker == 'Yes')),'totbill','tip')), vjust = -1)) +
        ggtitle("Female Smokers") +
        xlim(1,50) + ylim(1,10)

grid.arrange(p1, p2, p3, p4, ncol = 2, main = "Scatterplots with different sample subsets")

![](assets/unnamed-chunk-52-1.png)

Inspecting these plots reveals numerous features:

1. For smoking parties, there is a lot less association between the size of the tip and the size of the bill; 
1. When a female non-smoker paid the bill, the tip was a very consistent percentage of the bill, with the exceptions of three dining parties;
1. larger bills were mostly paid by men.

In the above example we gained a wealth of insight in a short time. Using nothing but graphical methods we investigated univariate, bivariate, and multivariate relationships. We found both global features and local detail. We saw that tips were rounded; then we saw the obvious correlation between the tip and the size of the bill, noting the scarcity of generous tippers; finally we discovered differences in the tipping behavior of male and female smokers and non-smokers. In contrast, an old-fashioned approach without graphics would be to fit a regression model. Without subtle regression diagnostics (which rely on graphics!), mmthis approach would miss many of the above insights.

## The Bigger picture

![](assets/images-mod1-big-picture-eda.gif)

To understand data analysis, and how visualization fits in, it is useful to talk about it as a process consisting of several stages:

#### The problem statement

Why do you want to analyze this data? Underlying every dataset is a question or problem statement. For the tipping data the question was provided to us from the data source: “What are the factors that affect tipping behavior?” This problem statement drives the process of any data analysis. Sometimes the problem is identified prior to a data collection. Perhaps it is realized after data becomes available because having the data available has made it possible to imagine new issues

#### Data preparation

During this class, I always hand you a single data matrix with each variable clearly defined. In the real world, it can take a great deal of work to construct a clean data matrix. For example, data values may be missing or misrecorded, data may be distributed across several sources, and the variable definitions and data values may be inconsistent across these sources. It is therefore not uncommon for this stage to consume most of the effort that goes into a project. And it is also not uncommon to loop back to this stage after completing the subsequent stages, to re-prepare and re-analyze the data.

#### Exploratory data analysis

At this stage in the analysis, we make time to “play in the sand” to allow us to find the unexpected, and come to some understanding of our data. It is characterized by direct manipulation and dynamic graphics: plots that respond in real time to an analyst’s queries and change dynamically to re-focus, link to information from other sources, and re-organize information. The analyst can work rapidly and thoroughly through the data, slipping out of dead-ends and chasing down new leads. The high level of interactivity is enabled by bare-bones graphics, which are generally not adequate for presentation purposes.

#### Quantitative analysis

At this stage, we use statistical modeling and statistical interference to answer our primary questions. With statistical models, we summarize complex data, decomposing it into estimates of signal and noise. With statistical inference, we try to assess whether a signal is real. Data visualization plays an important role at this stage, although that is less well known than its key role in exploration. It is helpful both in better understanding a model and in assessing its validity in relation to the data.

For Tips, we have not yet answered the primary question of interest. Let’s fit a regression model using tiprate as the response and the remaining variables (except tip and bill) as the explanatory variables. When we do this, only `size` has a significant regression coefficient. 

`ggplot` support for python is still under development, so for our linear model plots, let's use a tool called seaborn.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

#### Visualizing multiple regression with lmplot()

In [ ]:
with sns.plotting_context(context='poster'):
    sns.lmplot("totbill", "tip", tips, size=9, aspect=2)

In [ ]:
tips

This plot has two main components. The first is a scatterplot, showing the observed datapoints. The second is a regression line, showing the estimated linear model relating the two variables. Because the regression line is only an estimate, it is plotted with a 95% confidence band to give an impression of the certainty in the model. You can plot different levels of certainty

#### Plotting with discrete predictor variables

Sometimes you will want to plot data where the independent variable is quantitative, but discrete. Although this works fine out of the box:

In [ ]:
with sns.plotting_context(context='poster'):
    sns.lmplot("size", "tip", tips, size=6, aspect=2)

And can be improved with a bit of jitter:

In [ ]:
with sns.plotting_context(context='poster'):
    sns.lmplot("size", "tip", tips, size=6, aspect=2, x_jitter=.15);

It might be more informative to estimate the central tendency of each bin. This is easy to do with the x_estimator argument. Just pass any function that aggregates a vector of data into one estimate. The estimator will be bootstrapped and a confidence interval will be plotted – 95% by default, as in other cases within these functions.

In [ ]:
with sns.plotting_context(context='poster'):
    sns.lmplot("size", "tip", tips, x_estimator=np.mean, size=6, aspect=2);

### So what about the model?

Are we satisfied with this model? We have some doubts about it.

In [ ]:
tips['tiprate'] = tips.tip / tips.totbill

In [ ]:
len(tips[tips['tiprate'] < 0.18])

In [ ]:
from statsmodels.formula.api import ols

results = ols('tip ~ totbill', tips).fit()
results.summary()

However, in this data it explains only 2% of the variation in tip rate. The points are spread widely around the regression line. There are very few data points for parties of size one, five, and six, which makes us question the validity of the model in these regions. The signal is very weak relative to the noise.

### Presentation

Once an analysis has been completed, the results must be reported, either to clients, managers, or colleagues. The results probably take the form of a narrative and include quantitative summaries such as tables, forecasts, models, and graphics. Quite often, graphics form the bulk of the summaries.

Let's look at some other ways in which data can be sliced and diced in graphs

#### Faceted linear model plots

The lmplot() function is built on top of a FacetGrid. That means it’s easy to visualize how this relationship changes in different subsets of your dataset. You can read the extended documentation for more details on how the FacetGrid class works. The important thing is that you can supply the names of categorical variables that define subsets of the data to plot in different hues or along the row and columns of a grid of axes.

Using a hue facet makes it easiest to directly compare the two subsets:

In [ ]:
with sns.plotting_context(context='poster'):
    sns.lmplot("totbill", "tip", tips, hue="smoker", size=8, aspect=2);

To make plots that will better reproduce to black-and-white (i.e. when printed), you may want to let the scatterplot marker vary along with the hue variable.

`lmplot()` accepts all arguments that you would use to initialize a FacetGrid, and it returns the grid object after plotting for further tweaking:

In [ ]:
with sns.plotting_context(context='poster'):
    sns.lmplot("totbill", "tip", tips, hue="smoker", markers=["x", "o"], size=6, aspect=2);

Plotting in different columns of a grid also makes a plot that’s easy to understand, although direct comparisons between the subsets are more difficult as the data are separated in space. This might be better when you want the viewer to focus on the relationship within each subset independently.

In [ ]:
with sns.plotting_context(context='poster'):
    sns.lmplot("totbill", "tip", tips, col="smoker")

`lmplot()` accepts all arguments that you would use to initialize a FacetGrid, and it returns the grid object after plotting for further tweaking:

In [ ]:
sun_tips = tips[tips['day'] == 'Sun']

In [ ]:
with sns.plotting_context(context='poster'):
    g = sns.lmplot("totbill", "tip", sun_tips, hue="day", palette="Set2",
                   hue_order=["Sun"], size=8, aspect=2)

    g.set_axis_labels("Total bill (US Dollars)", "Tip");

    g.set(xticks=[10, 30, 50], ylim=(0, 10), yticks=[0, 2.5, 5, 7.5, 10]);

## Colofon

In [89]:
from utils import *
print_versions()

Python    2.7.10
IPython   4.0.0
numpy     1.10.1
pandas    0.17.0
sklearn   0.16.1
seaborn   0.6.0


In [90]:
%%html

<link rel="stylesheet" href="theme/custom.css">